# Workshop Notebook 3: Observability Part 1 - Validation Rules

In the previous notebooks you uploaded the models and artifacts, then deployed the models to production through provisioning workspaces and pipelines. Now you're ready to put your feet up! But to keep your models operational, your work's not done once the model is in production. You must continue to monitor the behavior and performance of the model to insure that the model provides value to the business.

In this notebook, you will learn about adding validation rules to pipelines.

## Preliminaries

In the blocks below we will preload some required libraries.

In [1]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework


from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json
import datetime
import time


## Login to Wallaroo

Retrieve the previous workspace, model versions, and pipelines used in the previous notebook.

In [2]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-cybersecurity"

workspace = wl.get_workspace(name=workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

model_name = 'aloha-prime'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'aloha-fraud-detector'

pipeline = wl.get_pipeline(pipeline_name)

# verify the workspace/pipeline/model

display(wl.get_current_workspace())
display(prime_model_version)
display(pipeline)



{'name': 'workshop-workspace-john-cybersecurity', 'id': 14, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-05-01T16:30:01.177583+00:00', 'models': [{'name': 'aloha-prime', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 1, 16, 30, 43, 651533, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 1, 16, 30, 43, 651533, tzinfo=tzutc())}, {'name': 'aloha-challenger', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 5, 1, 16, 38, 56, 600586, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 5, 1, 16, 38, 56, 600586, tzinfo=tzutc())}], 'pipelines': [{'name': 'aloha-fraud-detector', 'create_time': datetime.datetime(2024, 5, 1, 16, 30, 53, 995114, tzinfo=tzutc()), 'definition': '[]'}]}

Name,aloha-prime
Version,c719bc50-f83f-4c79-b4af-f66395a8da04
File Name,aloha-cnn-lstm.zip
SHA,fd998cd5e4964bbbb4f8d29d245a8ac67df81b62be767afbceb96a03d1a01520
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-01-May 16:30:43


name,aloha-fraud-detector
created,2024-05-01 16:30:53.995114+00:00
last_updated,2024-05-01 16:45:19.372610+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"551242a7-fe4c-4a61-a4c7-e7fcc97509dc, d22eb0d2-9cff-4c5f-a851-10b1a19d8c44, 262909e9-8779-4c56-a994-725ddd0b58c8, 4cdf8e11-1b9c-44ab-a16d-abb054b5e9fe, 6b3529b1-1ff1-454b-8896-460c8c90d667"
steps,aloha-prime
published,False


## Deploy the Pipeline

Add the model version as a pipeline step to our pipeline, and deploy the pipeline.  You may want to check the pipeline steps to verify that the right model version is set for the pipeline step.

Once deployed, perform a sample inference to make sure everythign is running smooth.

In [3]:
## blank space to get your pipeline and run a small batch of data through it to see the range of predictions

pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/data-1k.df.json')
display(multiple_result.sort_values(by=["out.cryptolocker"]).loc[:, ['out.cryptolocker']])


,out.cryptolocker
0,[0.012099549]
1,[0.025435215]
2,[0.036468606]
3,[0.100503676]
4,[0.0143616935]
...,...
995,[0.011983096]
996,[0.10195666]
997,[0.09424207]
998,[0.13612257]


## Model Validation Rules

A simple way to try to keep your model's behavior up to snuff is to make sure that it receives inputs that it expects, and that its output is something that downstream systems can handle. This can entail specifying rules that document what you expect, and either enforcing these rules (by refusing to make a prediction), or at least logging an alert that the expectations described by your validation rules have been violated. As the developer of the model, the data scientist (along with relevant subject matter experts) will often be the person in the best position to specify appropriate validation rules.

In our house price prediction example, suppose you know that house prices in your market are typically in the range $750,000 to $1.5M dollars. Then you might want to set validation rules on your model pipeline to specify that you expect the model's predictions to also be in that range. Then, if the model predicts a value outside that range, the pipeline will log that one of the validation checks has failed; this allows you to investigate that instance further.

Wallaroo provides **validations** to detect anomalous data from inference inputs and outputs.  Validations are added to a Wallaroo pipeline with the `wallaroo.pipeline.add_validations` method.

Adding validations takes the format:

```python
pipeline.add_validations(
    validation_name_01 = polars.col(in|out.{column_name}) EXPRESSION,
    validation_name_02 = polars.col(in|out.{column_name}) EXPRESSION
    ...{additional rules}
)
```

* `validation_name`: The user provided name of the validation.  The names must match Python variable naming requirements.
  * **IMPORTANT NOTE**: Using the name `count` as a validation name **returns a warning**.  Any validation rules named `count` are dropped upon request and an warning returned.
* `polars.col(in|out.{column_name})`: Specifies the **input** or **output** for a specific field aka "column" in an inference result.  Wallaroo inference requests are in the format `in.{field_name}` for **inputs**, and `out.{field_name}` for **outputs**.
  * More than one field can be selected, as long as they follow the rules of the [polars 0.18 Expressions library](https://docs.pola.rs/docs/python/version/0.18/reference/expressions/index.html).
* `EXPRESSION`:  The expression to validate. When the expression returns **True**, that indicates an anomaly detected.

The [`polars` library version 0.18.5](https://docs.pola.rs/docs/python/version/0.18/index.html) is used to create the validation rule.  This is installed by default with the Wallaroo SDK.  This provides a powerful range of comparisons to organizations tracking anomalous data from their ML models.

When validations are added to a pipeline, inference request outputs return the following fields:

| Field | Type | Description |
|---|---|---|
| **anomaly.count** | **Integer** | The total of all validations that returned **True**. |
| **anomaly.{validation name}** | **Bool** | The output of the validation `{validation_name}`. |

When validation returns `True`, **an anomaly is detected**.

For example, adding the validation `fraud` to the following pipeline returns `anomaly.count` of `1` when the validation `fraud` returns `True`.  The validation `fraud` returns `True` when the **output** field **dense_1** at index **0** is greater than 0.9.

```python
sample_pipeline = wallaroo.client.build_pipeline("sample-pipeline")
sample_pipeline.add_model_step(model)

# add the validation
sample_pipeline.add_validations(
    fraud=pl.col("out.dense_1").list.get(0) > 0.9,
    )

# deploy the pipeline
sample_pipeline.deploy()

# sample inference
display(sample_pipeline.infer_from_file("dev_high_fraud.json", data_format='pandas-records'))
```

|&nbsp;|time|in.tensor|out.dense_1|anomaly.count|anomaly.fraud|
|---|---|---|---|---|---|
|0|2024-02-02 16:05:42.152|[1.0678324729, 18.1555563975, -1.6589551058, 5...]|[0.981199]|1|True|

### Detecting Anomalies from Inference Request Results

When an inference request is submitted to a Wallaroo pipeline with validations, the following fields are output:

| Field | Type | Description |
|---|---|---|
| **anomaly.count** | **Integer** | The total of all validations that returned **True**. |
| **anomaly.{validation name}** | **Bool** | The output of each pipeline validation `{validation_name}`. |

For example, adding the validation `fraud` to the following pipeline returns `anomaly.count` of `1` when the validation `fraud` returns `True`.

```python
sample_pipeline = wallaroo.client.build_pipeline("sample-pipeline")
sample_pipeline.add_model_step(model)

# add the validation
sample_pipeline.add_validations(
    fraud=pl.col("out.dense_1").list.get(0) > 0.9,
    )

# deploy the pipeline
sample_pipeline.deploy()

# sample inference
display(sample_pipeline.infer_from_file("dev_high_fraud.json", data_format='pandas-records'))
```

|&nbsp;|time|in.tensor|out.dense_1|anomaly.count|anomaly.fraud|
|---|---|---|---|---|---|
|0|2024-02-02 16:05:42.152|[1.0678324729, 18.1555563975, -1.6589551058, 5...]|[0.981199]|1|True|

### Model Validation Rules Exercise

Add some simple validation rules to the model pipeline that you created in a previous exercise.

* Add an upper bound or a lower bound to the model predictions.
* Try to create predictions that fall both in and out of the specified range.
* Look through the logs to find anomalies.

**HINT 1**: since the purpose of this exercise is try out validation rules, it might be a good idea to take a small data set and make predictions on that data set first, *then* set the validation rules based on those predictions, so that you can see the check failures trigger.

Here's an example:

```python
import polars as pl

sample_pipeline = sample_pipeline.add_validations(
    too_low=pl.col("out.cryptolocker").list.get(0) < 0.4
)
```

In [5]:
# blank space to set a validation rule on the pipeline and check if it triggers as expected

import polars as pl

pipeline = pipeline.add_validations(
    too_low=pl.col("out.cryptolocker").list.get(0) < 0.1
)


In [6]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

multiple_result = pipeline.infer_from_file('../data/data-1k.df.json')

# show the first to results
display(multiple_result.head(20))


,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count,anomaly.too_low
0,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195814],[0.98291475],[0.012099549],[4.7591115e-05],[2.0289312e-05],[0.00031977257],[0.011029262],[0.997564],[0.010341609],[0.008038961],[0.016155047],[0.00623623],[0.0009985747],[1.7933434e-26],[1.388995e-27],1,True
1,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 27, 31, 29, 28, 15, 33, 29, 12, 36, 31, 12]",[2.837503e-05],[1.2753118e-05],[0.025435215],[6.150966e-10],[2.321774e-10],[0.051351104],[0.022038758],[0.9885122],[0.023624167],[0.017496044],[0.07612714],[0.018284446],[0.00016227343],[2.9736e-26],[6.570557e-23],1,True
2,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 25, 21, 16, 22, 20, 19, 19, 28]",[3.0770573e-07],[4.86675e-05],[0.036468606],[2.0693407e-15],[7.2607375e-18],[0.09667879],[0.073321395],[0.9993358],[0.0913113],[0.0527945],[2.7352993e-07],[0.041695543],[0.052203804],[4.6102867e-37],[3.6129874e-29],1,True
3,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 20, 22, 18, 32, 15, 12, 33, 17, 31, 14, 14, 27, 18]",[8.8228285e-13],[3.5226062e-06],[0.100503676],[1.6081854e-09],[3.923381e-17],[0.15465459],[0.24250229],[0.99999857],[0.25655058],[0.13984609],[2.9986824e-05],[0.16115357],[0.038542073],[2.5434677e-31],[5.6750776e-37],0,False
4,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 22, 12, 30, 24, 13, 19, 25, 36, 28, 13, 12, 13]",[5.4870607e-06],[0.0029785605],[0.0143616935],[1.9806076e-10],[3.051769e-10],[0.014699642],[0.03709711],[0.9984837],[0.036889926],[0.021504985],[0.0007605833],[0.017085439],[0.0009147275],[0.0],[8.360769e-30],1,True
5,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 18, 29, 25, 31, 21, 20, 19, 32, 30, 28, 12, 27, 34, 22, 29, 33]",[8.510186e-19],[1.9487171e-08],[0.112345986],[4.2180977e-07],[3.845866e-25],[0.08677925],[0.26976353],[1.0],[0.28034106],[0.1772562],[1.1434392e-08],[0.18945046],[0.031674143],[1.7841078e-30],[1.9434036e-36],0,False
6,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 27, 29, 19, 35, 27, 12, 15, 15, 33, 27, 27, 33, 33]",[8.3179456e-16],[2.1139394e-07],[0.10018868],[2.164278e-08],[3.9452582e-27],[0.06368506],[0.08174924],[0.9999998],[0.09102885],[0.05443449],[2.754149e-09],[0.06614307],[0.004162426],[2.3701194e-05],[3.0342645e-32],0,False
7,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 22, 34, 16, 33, 32, 35, 36, 28, 19, 32, 22, 35, 27]",[2.3569645e-18],[5.8529037e-10],[0.02562021],[1.3246556e-08],[9.615627e-30],[0.3749346],[0.17899293],[0.9999982],[0.21193951],[0.09348915],[2.2368522e-10],[0.123645365],[0.026871203],[7.0041276e-36],[0.0],1,True
8,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 23, 22, 17, 15, 25, 32, 31, 16, 28, 16, 22, 19, 31, 12, 23]",[1.051414e-09],[4.6342116e-08],[0.039195027],[2.8384156e-10],[3.1367284e-16],[0.11919281],[0.1371625],[0.9999869],[0.18870851],[0.08751174],[3.7058056e-05],[0.09724228],[0.012924853],[2.5838341e-33],[4.099651e-34],1,True
9,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,time,in.text_input,out.banjori,out.corebot,out.cryptolocker,out.dircrypt,out.gozi,out.kraken,out.locky,out.main,out.matsnu,out.pykspa,out.qakbot,out.ramdo,out.ramnit,out.simda,out.suppobox,anomaly.count,anomaly.too_low
0,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 28, 16, 32, 23, 29, 32, 30, 19, 26, 17]",[0.0015195814],[0.98291475],[0.012099549],[4.7591115e-05],[2.0289312e-05],[0.00031977257],[0.011029262],[0.997564],[0.010341609],[0.008038961],[0.016155047],[0.00623623],[0.0009985747],[1.7933434e-26],[1.388995e-27],1,True
1,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 27, 31, 29, 28, 15, 33, 29, 12, 36, 31, 12]",[2.837503e-05],[1.2753118e-05],[0.025435215],[6.150966e-10],[2.321774e-10],[0.051351104],[0.022038758],[0.9885122],[0.023624167],[0.017496044],[0.07612714],[0.018284446],[0.00016227343],[2.9736e-26],[6.570557e-23],1,True
2,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 25, 21, 16, 22, 20, 19, 19, 28]",[3.0770573e-07],[4.86675e-05],[0.036468606],[2.0693407e-15],[7.2607375e-18],[0.09667879],[0.073321395],[0.9993358],[0.0913113],[0.0527945],[2.7352993e-07],[0.041695543],[0.052203804],[4.6102867e-37],[3.6129874e-29],1,True
4,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 22, 12, 30, 24, 13, 19, 25, 36, 28, 13, 12, 13]",[5.4870607e-06],[0.0029785605],[0.0143616935],[1.9806076e-10],[3.051769e-10],[0.014699642],[0.03709711],[0.9984837],[0.036889926],[0.021504985],[0.0007605833],[0.017085439],[0.0009147275],[0.0],[8.360769e-30],1,True
7,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 22, 34, 16, 33, 32, 35, 36, 28, 19, 32, 22, 35, 27]",[2.3569645e-18],[5.8529037e-10],[0.02562021],[1.3246556e-08],[9.615627e-30],[0.3749346],[0.17899293],[0.9999982],[0.21193951],[0.09348915],[2.2368522e-10],[0.123645365],[0.026871203],[7.0041276e-36],[0.0],1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 30, 34, 19, 14, 12, 22, 29, 30, 22, 15, 23, 18, 22, 30, 31, 23]",[1.9509931e-08],[3.8211652e-05],[0.074470066],[1.5295203e-10],[4.951934e-13],[0.11857636],[0.23397556],[0.9999181],[0.26928952],[0.13449039],[0.00023073937],[0.15711878],[0.04463736],[0.0],[0.0],1,True
988,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 14, 32, 35, 25, 20, 31, 18, 26, 26, 35]",[0.00017462275],[2.0345014e-05],[0.086056456],[5.3362342e-08],[2.9563812e-06],[0.0585213],[0.14665367],[0.9945637],[0.12845428],[0.07318983],[0.048846442],[0.07254204],[0.09940087],[2.7488947e-32],[1.0424284e-31],1,True
995,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 31, 35, 13, 14, 25, 23, 14, 21, 20, 32, 14, 32, 29, 16, 33]",[2.0416806e-12],[7.744161e-09],[0.011983096],[8.3120476e-08],[7.3397146e-14],[0.123229906],[0.07365251],[0.9999754],[0.10082034],[0.057467848],[0.00017084264],[0.07321706],[0.0018467624],[3.284046e-36],[0.0],1,True
997,2024-05-01 18:56:42.249,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 28, 19, 19, 28, 15, 12, 18]",[0.011286308],[0.06214513],[0.09424207],[6.8554013e-16],[0.0031474212],[0.013521446],[0.0521531],[0.66066873],[0.039717037],[0.026172414],[0.015154914],[0.019907918],[0.040248252],[9.4758866e-27],[1.1339277e-21],1,True


In [8]:
# show only the results that triggered an anomaly
multiple_result.loc[multiple_result['anomaly.count'] > 0, ['time', 'out.cryptolocker']]

,time,out.cryptolocker
0,2024-05-01 18:56:42.249,[0.012099549]
1,2024-05-01 18:56:42.249,[0.025435215]
2,2024-05-01 18:56:42.249,[0.036468606]
4,2024-05-01 18:56:42.249,[0.0143616935]
7,2024-05-01 18:56:42.249,[0.02562021]
...,...,...
985,2024-05-01 18:56:42.249,[0.074470066]
988,2024-05-01 18:56:42.249,[0.086056456]
995,2024-05-01 18:56:42.249,[0.011983096]
997,2024-05-01 18:56:42.249,[0.09424207]


## Clean Up

At this point, if you are not continuing on to the next notebook, undeploy your pipeline to give the resources back to the environment.

In [9]:
## blank space to undeploy the pipeline

pipeline.undeploy()

name,aloha-fraud-detector
created,2024-05-01 16:30:53.995114+00:00
last_updated,2024-05-01 18:56:40.187633+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"28f5443a-ff67-4f4f-bfc3-f3f95f3c6f83, 435b73f7-76a1-4514-bd41-2cb94d3e78ff, 551242a7-fe4c-4a61-a4c7-e7fcc97509dc, d22eb0d2-9cff-4c5f-a851-10b1a19d8c44, 262909e9-8779-4c56-a994-725ddd0b58c8, 4cdf8e11-1b9c-44ab-a16d-abb054b5e9fe, 6b3529b1-1ff1-454b-8896-460c8c90d667"
steps,aloha-prime
published,False


## Congratulations!

In this workshop you have

* Set a validation rule on your house price prediction pipeline.
* Detected model predictions that failed the validation rule.

In the next notebook, you will learn how to monitor the distribution of model outputs for drift away from expected behavior.